In [1]:
import sys
import gc

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
org_train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv", sep=',')

In [3]:
train.rename(columns={'label':'generated'},inplace=True)
train.drop(columns=['prompt_name','source','RDizzl3_seven'],inplace=True)

In [4]:
train_new_a=pd.read_csv("/kaggle/input/10-74k-detect-ai-generated-text/essays_a.csv")
train_new_b=pd.read_csv("/kaggle/input/10-74k-detect-ai-generated-text/essays_b.csv")
train_new_c=pd.read_csv("/kaggle/input/10-74k-detect-ai-generated-text/essays_c.csv")
train_new_d=pd.read_csv("/kaggle/input/10-74k-detect-ai-generated-text/essays_d.csv")


In [5]:
train_new_a.drop(columns=['prompt_name','prompt_id'],inplace=True)
train_new_b.drop(columns=['prompt_name','prompt_id'],inplace=True)
train_new_c.drop(columns=['prompt_name','prompt_id'],inplace=True)

In [6]:
train=pd.concat([train_new_a,train_new_b,train_new_c,train_new_d,train])

In [7]:
train = train.drop_duplicates(subset=['text'])

train.reset_index(drop=True, inplace=True)

In [8]:
LOWERCASE = False
VOCAB_SIZE = 30522

In [10]:
# Creating Byte-Pair Encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
# Adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)
# Creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])
def train_corp_iter(): 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

import pickle
with open('raw_vectorizer.pk', 'wb') as fin:
    pickle.dump(raw_tokenizer, fin)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/55608 [00:00<?, ?it/s]

In [ ]:
def dummy(text):
    return text

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, analyzer = 'word',
    tokenizer = dummy,
    preprocessor = dummy,
    token_pattern = None, strip_accents='unicode'
                            )

vectorizer.fit(tokenized_texts_test)
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
import pickle
with open('vectorizer.pk', 'wb') as fin:
    pickle.dump(vectorizer, fin)
# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode'
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
with open('vectorizer_1.pk', 'wb') as fina:
    pickle.dump(vectorizer, fina)
tf_test = vectorizer.transform(tokenized_texts_test)

del vectorizer
gc.collect()


In [ ]:
y_train = train['generated'].values

In [ ]:
clf = MultinomialNB(alpha=0.02)
clf2 = MultinomialNB(alpha=0.01)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
p6={'n_iter': 1000,'verbose': -1,'learning_rate': 0.005689066836106983, 'colsample_bytree': 0.8915976762048253, 'colsample_bynode': 0.5942203285139224, 'lambda_l1': 7.6277555139102864, 'lambda_l2': 6.6591278779517808, 'min_data_in_leaf' : 156, 'max_depth': 11, 'max_bin': 813}
lgb=LGBMClassifier(**p6)

ensemble = VotingClassifier(estimators=[('mnb',clf),
                                        ('sgd', sgd_model),
                                        ('lgb',lgb)
                                       ],
                            weights=[0.2,0.4,0.4], voting='soft', n_jobs=-1)
ensemble.fit(tf_train, y_train)

gc.collect()
import pickle
filename = 'finalized_model.sav'
pickle.dump(ensemble, open(filename, 'wb'))